# KOSMI 2024 Winter School
## Session 4. 나만의 거대언어모델 만들어 보기

- 연자: 김준우(kjune0322@kaist.ac.kr), 권순준(sean0042@kaist.ac.kr)
- 발표자료: https://github.com/starmpcc/KOSMI2024-Asclepius



<a target="_blank" href="https://colab.research.google.com/github/starmpcc/KOSMI2024-Asclepius/blob/main/KOSMI_LLM_Inst_FT.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# First, install required packages
!pip install -q accelerate==0.25.0 peft==0.6.2 bitsandbytes==0.41.1 transformers==4.36.2 trl==0.7.4 einops gradio openai

In [2]:
# Import Libraries
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import gradio as gr

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
# To save time, first download model and data

# Define Quantization Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

# Load Model and Dataset
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
    revision="refs/pr/23"
)

tokenizer = AutoTokenizer.from_pretrained('microsoft/phi-2')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_sight = "right"

def prompt_shorter_than(samples):
    concatenated = [" ".join([i, j, k]) for i, j, k in zip(samples['note'], samples['question'], samples['answer'])]
    return [len(i)<=320 for i in tokenizer(concatenated)['input_ids']]

dataset = load_dataset("starmpcc/Asclepius-Synthetic-Clinical-Notes")
dataset = dataset.filter(lambda x: [len(i)<1500 for i in x['note']], batched=True)
dataset = dataset.filter(prompt_shorter_than, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Let's pre-process dataset

# First of all, we have to check how the dataset is composed
print(dataset['train'])
dataset['train'][0]

Dataset({
    features: ['patient_id', 'note', 'question', 'answer', 'task'],
    num_rows: 3686
})


{'patient_id': 110,
 'note': "Hospital Course:\n\nThe patient, who was involved in a murder case, was admitted to our forensic facility for toxicology testing and a complete neuropsychiatric evaluation. The case is complicated due to the defendant's claimed genetic predisposition to anti-social behavior and his regular consumption of alcohol and drugs since the beginning of adolescence.\n\nToxicology testing revealed detectable levels of benzoylecgonine in urine and pubic hair, while blood and saliva samples showed no significant levels of drugs or alcohol. A full clinical and neuropsychological examination was performed, which identified a personality disorder not otherwise specified. MRI imaging showed a decrease in cortical thickness with enlarged lateral ventricles, significant volumetric asymmetry of the amygdalae, and a decreased volume of the right orbito-frontal cortex in comparison with the left one. PET-CT testing did not indicate any alteration of brain perfusion or metaboli

In [5]:
# We make this dataset to phi-2 compatible
# Phi-2 instruction-answer format: "Instruct: <prompt>\nOutput:"

# Make your own prompt!
prompt_template="""Instruct: Please write down your own prompt.
For instance, you can insert the note as {{note}}
{note}
Model should answer to {{question}} based on the note.
{question}
You should maintain the phi-2 format
Accordingly, the last line must be like the below.
Do not forget to insert a new line between your prompt and 'Output'!
Output: {answer}
"""


# Should get Dict[List] as input, return list of prompts
def format_dataset(samples):
    outputs = []
    for _, note, question, answer, _ in zip(*samples.values()):
        out = prompt_template.format(note=note, question=question, answer=answer)
        outputs.append(out)
    return outputs

sample_input = format_dataset({k: [v] for k, v in dataset['train'][0].items()})[0]
print(sample_input)
print("*"*20)

# Sanity Check
prompt_len = len(tokenizer.encode(prompt_template))
if prompt_len > 180:
    raise ValueError(f"Your prompt is too long! Please reduce the length from {prompt_len} to 180 tokens")
print(f"Prompt Length: {prompt_len} tokens")

Instruct: Please write down your own prompt.
For instance, you can insert the note as {note}
Hospital Course:

The patient, who was involved in a murder case, was admitted to our forensic facility for toxicology testing and a complete neuropsychiatric evaluation. The case is complicated due to the defendant's claimed genetic predisposition to anti-social behavior and his regular consumption of alcohol and drugs since the beginning of adolescence.

Toxicology testing revealed detectable levels of benzoylecgonine in urine and pubic hair, while blood and saliva samples showed no significant levels of drugs or alcohol. A full clinical and neuropsychological examination was performed, which identified a personality disorder not otherwise specified. MRI imaging showed a decrease in cortical thickness with enlarged lateral ventricles, significant volumetric asymmetry of the amygdalae, and a decreased volume of the right orbito-frontal cortex in comparison with the left one. PET-CT testing did

In [6]:
sample_idx = 0
sample_input = format_dataset({k: [v] for k, v in dataset['train'][sample_idx].items()})[0].split('Output: ')[0]
input_ids = tokenizer.encode(sample_input, return_tensors='pt').to('cuda')
with torch.no_grad():
  output = model.generate(input_ids=input_ids,
                            max_length=512,
                            use_cache=True,
                            temperature=0.,
                            eos_token_id=tokenizer.eos_token_id
  )
print(tokenizer.decode(output.to('cpu')[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruct: Please write down your own prompt.
For instance, you can insert the note as {note}
Hospital Course:

The patient, who was involved in a murder case, was admitted to our forensic facility for toxicology testing and a complete neuropsychiatric evaluation. The case is complicated due to the defendant's claimed genetic predisposition to anti-social behavior and his regular consumption of alcohol and drugs since the beginning of adolescence.

Toxicology testing revealed detectable levels of benzoylecgonine in urine and pubic hair, while blood and saliva samples showed no significant levels of drugs or alcohol. A full clinical and neuropsychological examination was performed, which identified a personality disorder not otherwise specified. MRI imaging showed a decrease in cortical thickness with enlarged lateral ventricles, significant volumetric asymmetry of the amygdalae, and a decreased volume of the right orbito-frontal cortex in comparison with the left one. PET-CT testing did

In [7]:
# Then, let's define dataset.
response_template = "Output:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

train_dataset = dataset['train']
sampled_train_dataset = train_dataset.select(range(2000))

In [8]:
# SFTTrainer Do everything else for you!

lora_config=LoraConfig(
    r=4,
    task_type="CAUSAL_LM",
    target_modules= ["Wqkv", "fc1", "fc2" ]
)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    fp16=True,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    optim="paged_adamw_32bit",
    save_strategy="no",
    warmup_ratio=0.03,
    logging_steps=5,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    gradient_checkpointing=True
)

trainer = SFTTrainer(
    model,
    training_args,
    train_dataset=sampled_train_dataset,
    formatting_func=format_dataset,
    data_collator=collator,
    peft_config=lora_config,
    max_seq_length=512,
    tokenizer=tokenizer,
)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [ ]:
# Run Training
trainer.train()

You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,0.700500
10,0.653100
15,0.604900
20,0.550800
25,0.486300
30,0.465200
35,0.512800
40,0.601100
45,0.579700
50,0.512100


TrainOutput(global_step=125, training_loss=0.5334161357879639, metrics={'train_runtime': 1388.8048, 'train_samples_per_second': 1.44, 'train_steps_per_second': 0.09, 'total_flos': 1.237657008494592e+16, 'train_loss': 0.5334161357879639, 'epoch': 1.0})

In [ ]:
# Wrap-up Training
model = trainer.model
model.eval()

note_samples = train_dataset.select(range(len(train_dataset)-10, len(train_dataset)))['note']

def inference(note, question, model):
    prompt = prompt_template.format(note=note, question=question, answer="")
    tokens = tokenizer.encode(prompt, return_tensors="pt").to('cuda')
    outs = model.generate(input_ids=tokens,
                          max_length=512,
                          use_cache=True,
                          temperature=0.,
                          eos_token_id=tokenizer.eos_token_id
                          )
    output_text = tokenizer.decode(outs.to('cpu')[0], skip_special_tokens=True)
    return output_text[len(prompt):]


def compare_models(note, question):
    with torch.no_grad():
        asc_answer = inference(note, question, trainer.model)
        with model.disable_adapter():
            phi_answer = inference(note, question, trainer.model)
    return asc_answer, phi_answer

demo = gr.Interface(fn=compare_models, inputs=[gr.Dropdown(note_samples), "text"], outputs=[gr.Textbox(label="Asclepius"), gr.Textbox(label="Phi-2")])
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://86d6f91a8d7f0bfa0d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
